### Passo 1 - Importar Arquivos e Bibliotecas

In [14]:
# Importar bibliotecas
import pandas as pd
import pathlib

from googleapiclient.discovery import build
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from google_auth_oauthlib.flow import InstalledAppFlow
import pybase64

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
# Importar bases de dados

emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [16]:
# Incluir nome da loja no database de vendas
vendas = vendas.merge(lojas, on='ID Loja')

# Dicionário com o nome das lojas
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

# Definir o dia do indicador - Última data do relatório
dia_indicador = vendas['Data'].max()
print(dia_indicador)
print('{}/{}'.format(dia_indicador.day, dia_indicador.month))

2019-12-26 00:00:00
26/12


### Passo 3 - Salvar a planilha na pasta de backup

In [17]:
#identificar se a pasta com o bakcup dos arquivos das Lojas já existe
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

In [18]:
# Criar diretório e gravar relatórios das lojas
for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    # Salvar arquivo dentro da pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Calcular o indicador para 1 loja

In [19]:
# Definição de metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [20]:
# Carregue as credenciais do Google API
creds = None
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify', 'https://www.googleapis.com/auth/gmail.compose']


# Carrega as credeciais - Roda novamente cada vez que o programa é disparado
credentials_file = 'credenciais.json' # Utilize suas próprias credenciais (https://console.cloud.google.com/apis/dashboard)
flow = InstalledAppFlow.from_client_secrets_file(
            credentials_file, SCOPES)
creds = flow.run_local_server(port=0)

# Crie um objeto de serviço para a API do Gmail
service = build('gmail', 'v1', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=831120541196-jlier2o198pf4h9707j6islc37s2i38o.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59638%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.compose&state=9TPG7MB25J94h2HSPRtjhLCKSDBZcy&access_type=offline


In [21]:
# Prepara e enviar os e-mails para cada loja
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    # Faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # Diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket medio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()

    # Ticket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()

    ### CRIAR E-MAIL DO GMAIL ###
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]

    # Prepare o conteúdo do e-mail
    remetente = 'thiagokato@gmail.com'
    destinatario = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    assunto = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    # Crie a mensagem MIME
    mensagem = MIMEMultipart()
    mensagem['From'] = remetente
    mensagem['To'] = destinatario
    mensagem['Subject'] = assunto
    
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    corpo_html = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., GERENTE</p>
    '''

    # Adicione a parte HTML do corpo da mensagem
    parte_html = MIMEText(corpo_html, 'html')
    mensagem.attach(parte_html)

    # Anexos:
    attachment  = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    anexo = MIMEBase('application', 'octet-stream')
    with open(attachment, 'rb') as arquivo:
        anexo.set_payload(arquivo.read())
        anexo.add_header('Content-Disposition', 'attachment; filename={}'.format(attachment))
    mensagem.attach(anexo)

    # Converta a mensagem em uma string codificada em Base64
    mensagem_bytes = mensagem.as_bytes()
    mensagem_codificada = pybase64.b64encode(mensagem_bytes).decode('utf-8')

    # Envie o e-mail
    requisicao = {
        'raw': mensagem_codificada
    }
    service.users().messages().send(userId='me', body=requisicao).execute()

    print('E-mail da Loja {} enviado'.format(loja))
    

E-mail da Loja Iguatemi Esplanada enviado
E-mail da Loja Shopping Midway Mall enviado
E-mail da Loja Norte Shopping enviado
E-mail da Loja Shopping Iguatemi Fortaleza enviado
E-mail da Loja Shopping União de Osasco enviado
E-mail da Loja Shopping Center Interlagos enviado
E-mail da Loja Rio Mar Recife enviado
E-mail da Loja Salvador Shopping enviado
E-mail da Loja Rio Mar Shopping Fortaleza enviado
E-mail da Loja Shopping Center Leste Aricanduva enviado
E-mail da Loja Ribeirão Shopping enviado
E-mail da Loja Shopping Morumbi enviado
E-mail da Loja Parque Dom Pedro Shopping enviado
E-mail da Loja Bourbon Shopping SP enviado
E-mail da Loja Palladium Shopping Curitiba enviado
E-mail da Loja Passei das Águas Shopping enviado
E-mail da Loja Center Shopping Uberlândia enviado
E-mail da Loja Shopping Recife enviado
E-mail da Loja Shopping Vila Velha enviado
E-mail da Loja Shopping SP Market enviado
E-mail da Loja Shopping Eldorado enviado
E-mail da Loja Shopping Ibirapuera enviado
E-mail da L

### Passo 5 - Criar ranking para diretoria

In [22]:
# Cria arquivos para envio à diretoria
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

### Passo 6 - Enviar e-mail para diretoria

In [23]:
# Prepare o conteúdo do e-mail
remetente = 'thiagokato@gmail.com'
destinatario = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
assunto = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'

# Crie a mensagem MIME
mensagem = MIMEMultipart()
mensagem['From'] = remetente
mensagem['To'] = destinatario
mensagem['Subject'] = assunto


corpo_html = f'''
<p>Prezados, bom dia</p>
<p></p>
<p>Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 0]:.2f}</p>
<p>Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 0]:.2f}</p>
<p></p>
<p>Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 0]:.2f}</p>
<p>Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 0]:.2f}</p>
<p></p>
<p>Segue em anexo os rankings do ano e do dia de todas as lojas.</p>
<p></p>
<p>Qualquer dúvida estou à disposição.</p>
<p></p>
<p>Att.,</p>
<p>GERENTE</p>
'''

# Adicione a parte HTML do corpo da mensagem
parte_html = MIMEText(corpo_html, 'html')
mensagem.attach(parte_html)

# Anexos:
attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
anexo = MIMEBase('application', 'octet-stream')
with open(attachment, 'rb') as arquivo:
    anexo.set_payload(arquivo.read())
    anexo.add_header('Content-Disposition', 'attachment; filename={}'.format(attachment))
mensagem.attach(anexo)

attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
anexo = MIMEBase('application', 'octet-stream')
with open(attachment, 'rb') as arquivo:
    anexo.set_payload(arquivo.read())
    anexo.add_header('Content-Disposition', 'attachment; filename={}'.format(attachment))
mensagem.attach(anexo)


# Converta a mensagem em uma string codificada em Base64
mensagem_bytes = mensagem.as_bytes()
mensagem_codificada = pybase64.b64encode(mensagem_bytes).decode('utf-8')

# Envie o e-mail
requisicao = {
    'raw': mensagem_codificada
}
service.users().messages().send(userId='me', body=requisicao).execute()

print('E-mail da Diretoria enviado')

E-mail da Diretoria enviado
